In [1]:
#update to be relative to your project structure
main_input_folder_path = r"C:\Users\webbb\VolkerWessels UK\Grp_VMS GeoSeacore Collaboration Space - SZC\2 Internal WIP\06. MST's\Weather Windows"

In [2]:
import sys
import os
import pandas as pd

# Add the src directory to the system path and import functions
sys.path.append(os.path.abspath('../src'))
from schedule import generate_activity_list, schedule_activities, sheduled_df, shift_start_end
from plot import plot

In [3]:
#read in activity data from excel
path = os.path.join(main_input_folder_path, "WeatherWindows.xlsx")
act_df = pd.read_excel(path, sheet_name="Sheet1", skiprows=1)
act_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 22 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   ID                                         54 non-null     object 
 1   Sub Activity                               54 non-null     object 
 2   Predecessor ID(s)                          52 non-null     object 
 3   Group                                      54 non-null     object 
 4   Phase                                      39 non-null     object 
 5   Reference Doc                              28 non-null     object 
 6   Resources required (Vessels, plants etc.)  38 non-null     object 
 7   Type                                       3 non-null      object 
 8   Duration (hours)                           54 non-null     float64
 9   Contingency time  (hours)                  0 non-null      float64
 10  Minimum Weather Window (hour

In [ ]:
#Example without weather

act_list = generate_activity_list(act_df) #generate activity list

scheduled_activities = schedule_activities(act_list) #schedule activities

scheduled_df = sheduled_df(scheduled_activities) #convert to dataframe for easier handling

schedule_activities = shift_start_end(scheduled_df) #shift to make punch out 0
scheduled_df.head()

,ID,Description,Duration (hours),Start (hours),End (hours),Group,Predecessor IDs
0,PM01,Mobilisation of Leask marine spread,5.0,-6.0,-1.0,Pipe Management,[]
1,CTV01,CTV transfer of Personnel to JUB,2.0,-6.0,-4.0,Crew Transfer Vessel,[]
2,HDD01,Punch out of pilot,3.0,0.0,3.0,Horizontal Directional Drilling,"[PM02, JUB01]"
3,CTV02,Bathymetric survey of pilot to confirm bore lo...,3.0,3.0,6.0,Crew Transfer Vessel,[HDD01]
4,HDD02,Pull back of pilot,6.0,6.0,12.0,Horizontal Directional Drilling,[CTV02]


In [ ]:
import pandas as pd

# Load activities
act_df = pd.read_excel(path, sheet_name="Sheet1", skiprows=1)
activities = generate_activity_list(act_df)

# Load weather/tide data
weather_df = pd.read_csv(
    r"C:\Users\webbb\VolkerWessels UK\Grp_VMS GeoSeacore Collaboration Space - SZC\2 Internal WIP\06. MST's\Weather Windows\scripts\combined_tide_data_with_rate.csv"
)

# Parse Time column
weather_df['Time'] = pd.to_datetime(weather_df['Time'])

# Ensure absolute rate for current proxy
weather_df['Tide_rate_m_per_hr_abs'] = weather_df['Tide_rate_m_per_hr'].abs()

# Define start date for hour numbering
start_date = weather_df['Time'].min()
weather_df['Hour_index'] = (weather_df['Time'] - start_date).dt.total_seconds() / 3600

# Build weather_data dictionary for scheduling
weather_data = {
    int(round(row['Hour_index'] / 0.5)): {  # 0.5 for 30-min interval
        "tidal_level": row['Tide Prediction (m)'],
        "tidal_current": row['Tide_rate_m_per_hr_abs']
    }
    for _, row in weather_df.iterrows()
}

# Schedule with 30-minute intervals
scheduled = schedule_activities(activities, weather_data, analysis_interval=0.5)
df = scheduled_df(scheduled)
df = shift_start_end(df)

print(df.head())

In [6]:
fig = plot(scheduled_df)

outputpath=os.path.join(main_input_folder_path, "pipe_pull_schedule.html")
fig.write_html(outputpath, include_plotlyjs='cdn')